In [3]:
import pandas as pd

# Load the dataset
file_name = "sff_somaliFootball_historical_data.csv"
df = pd.read_csv(file_name)

# Clean the date column (remove leading/trailing spaces)
df["date"] = df["date"].str.strip()

# Convert date to datetime format
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y", errors="coerce")

# Sort the dataset by date (ascending order)
df = df.sort_values(by="date")

# Extract Home_Score and Away_Score
df[["Home_Score", "Away_Score"]] = df["score"].str.split(" : ", expand=True).astype(int)

# Calculate Home_Goal_Diff and Away_Goal_Diff
df["Home_Goal_Diff"] = df["Home_Score"] - df["Away_Score"]
df["Away_Goal_Diff"] = df["Away_Score"] - df["Home_Score"]

# Determine Result
df["Result"] = df["Home_Goal_Diff"].apply(lambda x: "Win" if x > 0 else "Loss" if x < 0 else "Draw")

# Extract Season from date
df["Season"] = df["date"].dt.year

# Initialize new features
df["Home_Last_5_Wins"] = 0
df["Away_Last_5_Wins"] = 0

# Function to calculate last 5 wins for a team
def calculate_last_5_wins(team, current_date):
    # Filter matches for the team before the current date
    matches = df[(df["date"] < current_date) & 
                 ((df["home"] == team) | (df["away"] == team))]
    # Get the last 5 matches
    last_5_matches = matches.tail(5)
    # Calculate wins
    wins = len(last_5_matches[((last_5_matches["home"] == team) & (last_5_matches["Result"] == "Win")) | 
                              ((last_5_matches["away"] == team) & (last_5_matches["Result"] == "Loss"))])
    return wins

# Calculate features for each row
for i, row in df.iterrows():
    home_team = row["home"]
    away_team = row["away"]
    current_date = row["date"]
    
    # Calculate Home_Last_5_Wins
    df.at[i, "Home_Last_5_Wins"] = calculate_last_5_wins(home_team, current_date)
    
    # Calculate Away_Last_5_Wins
    df.at[i, "Away_Last_5_Wins"] = calculate_last_5_wins(away_team, current_date)

# Select and reorder columns
new_columns = [
    "Season", "date", "home", "away", "Home_Score", "Away_Score", 
    "Home_Goal_Diff", "Away_Goal_Diff", "Result", "Home_Last_5_Wins", "Away_Last_5_Wins"
]
df = df[new_columns]

# Rename columns for clarity
df.columns = [
    "Season", "Date", "Home_Team", "Away_Team", "Home_Score", "Away_Score", 
    "Home_Goal_Diff", "Away_Goal_Diff", "Result", "Home_Last_5_Wins", "Away_Last_5_Wins"
]

# Save to new CSV file
output_file_name = "Last_somali_football_data.csv"
df.to_csv(output_file_name, index=False)

print("Transformation complete! File saved as:", output_file_name)
print(df.head())  # Preview the transformed dataset


Transformation complete! File saved as: Last_somali_football_data.csv
   Season       Date              Home_Team      Away_Team  Home_Score  \
0    2012 2012-06-04              Elman FC    Badbaado FC            3   
1    2012 2012-06-05             Sahafi FC     Jeenyo  FC            1   
2    2012 2012-06-07            Heegan S.C     Dekedda SC            1   
3    2012 2012-06-08   Mogadishu City Club    Horseed S.C            1   
4    2012 2012-06-11            Jeenyo  FC       Elman FC            1   

   Away_Score  Home_Goal_Diff  Away_Goal_Diff Result  Home_Last_5_Wins  \
0           1               2              -2    Win                 0   
1           1               0               0   Draw                 0   
2           1               0               0   Draw                 0   
3           1               0               0   Draw                 0   
4           1               0               0   Draw                 0   

   Away_Last_5_Wins  
0                 